In [8]:
import cv2
import numpy as np
import os
import sys
from PIL import Image
from glob import glob
from tqdm import tqdm   
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        

In [11]:
folder_list=glob('../../data/stroke_CT/**/**/')
for i in tqdm(range(len(folder_list))):
    folder=folder_list[i]

    create_dir(folder.replace('stroke_CT','stroke_CT_preprocessed'))
    img_list=glob(folder+'*.png')
    for j in range(len(img_list)):
        img_path=img_list[j]
        img=Image.open(img_path)
        img=img.resize((512,512))
        img.save(img_path.replace('stroke_CT','stroke_CT_preprocessed'))

        


100%|██████████| 2435/2435 [33:54<00:00,  1.20it/s]
